<a href="https://colab.research.google.com/github/MichaelGuaca/Proyecto-IA/blob/main/04_modelo_soluci%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Importar librerías necesarias
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV


In [2]:
#IMPORTAR DATOS DE KAGGLE A COLAB
os.environ['KAGGLE_CONFIG_DIR'] = '.'
!chmod 600 ./kaggle.json
!kaggle competitions download -c udea-ai4eng-20242

 45% 9.00M/20.1M [00:00<00:00, 62.9MB/s]
100% 20.1M/20.1M [00:00<00:00, 91.4MB/s]


In [3]:
#DESCOMPRIMIR ARCHIVO
!unzip udea-ai4eng-20242.zip

Archive:  udea-ai4eng-20242.zip
  inflating: submission_example.csv  
  inflating: test.csv                
  inflating: train.csv               


In [4]:
#CARGAR DATOS TRAIN CON PANDAS
train_data = pd.read_csv("train.csv")

In [5]:
#Explorar datos
print(train_data.info())
print(train_data['RENDIMIENTO_GLOBAL'].value_counts())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 692500 entries, 0 to 692499
Data columns (total 12 columns):
 #   Column                          Non-Null Count   Dtype 
---  ------                          --------------   ----- 
 0   ID                              692500 non-null  int64 
 1   PERIODO                         692500 non-null  int64 
 2   ESTU_PRGM_ACADEMICO             692500 non-null  object
 3   ESTU_PRGM_DEPARTAMENTO          692500 non-null  object
 4   ESTU_VALORMATRICULAUNIVERSIDAD  686213 non-null  object
 5   ESTU_HORASSEMANATRABAJA         661643 non-null  object
 6   FAMI_ESTRATOVIVIENDA            660363 non-null  object
 7   FAMI_TIENEINTERNET              665871 non-null  object
 8   FAMI_EDUCACIONPADRE             669322 non-null  object
 9   FAMI_EDUCACIONMADRE             668836 non-null  object
 10  ESTU_PAGOMATRICULAPROPIO        686002 non-null  object
 11  RENDIMIENTO_GLOBAL              692500 non-null  object
dtypes: int64(2), object(10)
memory

In [6]:
#Preprocesamiento de datos
#Manejar valores faltantes (rellenando con una categoría específica para columnas categóricas)
for col in train_data.columns:
    if train_data[col].dtype == 'object':
        train_data[col].fillna('Desconocido', inplace=True)
    else:
        train_data[col].fillna(train_data[col].median(), inplace=True)

<ipython-input-6-73dd2afa7a95>:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_data[col].fillna(train_data[col].median(), inplace=True)
<ipython-input-6-73dd2afa7a95>:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True

In [7]:
#Codificar variables categóricas usando LabelEncoder
categorical_cols = train_data.select_dtypes(include='object').columns
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    train_data[col] = le.fit_transform(train_data[col])
    label_encoders[col] = le

In [8]:
#Separar características (X) y objetivo (y)
X = train_data.drop(['RENDIMIENTO_GLOBAL', 'ID', 'PERIODO'], axis=1)
y = train_data['RENDIMIENTO_GLOBAL']

In [9]:
#Dividir los datos en entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
#Definir la cuadrícula de hiperparámetros
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

In [11]:
#Inicializar el modelo
rf = RandomForestClassifier(random_state=42)

In [12]:
#Configurar GridSearchCV
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, n_jobs=-1, scoring='accuracy', verbose=2)


In [ ]:
#Ajustar el modelo a los datos de entrenamiento
grid_search.fit(X_train, y_train)


Fitting 3 folds for each of 162 candidates, totalling 486 fits


In [ ]:
#Mostrar los mejores hiperparámetros
print("Mejores hiperparámetros:", grid_search.best_params_)


In [ ]:
#Usar el mejor modelo para hacer predicciones
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_val)
print("Accuracy después de ajuste:", accuracy_score(y_val, y_pred))


In [ ]:
# Crear la carpeta .kaggle y mover el archivo json
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# Cambiar los permisos del archivo
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions submit -c udea-ai4eng-20242 -f submission.csv -m "Segunda Predicción"
